In [ ]:
# Let's build our project

# Here we are building a mulitple input mixed data model that takes in the 3 inputs seperately. RBI, chaneel matrices remain 
# seperate but now instead of giving Spectrum Occupancy matrix as input we give only NSA and instead of Src Des matrix we 
# just give Src and Des node id. The NSA and Src_Des input are given in two different branches, in the next model they will 
# be given together on one branch. The output power are indices rather matrices. So its an Nx1 vector.
!pip install tensorflow
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(tf.config.list_physical_devices())

# number of iterations the matlab code ran or the number of times the network is simulated
itr = 108090
# Number of iterations the test data generation matlab code ran or the number of times the network is simulated
itr_test = 7008
# number of nodes in the network
N = 30
# We scale the non zero values of output power so that there is some sizable error if the model outputs a zero matrix
scaling_parameter = 1

# Reading the RBI data
df = pd.read_csv("Data_rbi_vectors.csv")
#print(df)
df.shape

In [ ]:
# coverting it into numpy array
tem = df.to_numpy()
#print(tem)
tem.shape

#Just for the 1332 dataset as the last array will be nans
#tem = tem[:-1]

In [ ]:
a=list(range(0, itr))
X_1 = np.zeros((itr,N))
for i in a:
    X_1[i:] = tem[i:]
#X_1

In [ ]:
# Reading the channel data
df1 = pd.read_csv("Data_channel_matrices/Data_channel_matrices.csv")
#print(df1)
df1.shape

In [ ]:
# coverting it into numpy array
X_2 = df1.to_numpy()

In [ ]:
X_2 = X_2.reshape((78092,N,N))
#print(X_2)
X_2 = np.nan_to_num(X_2)
#X_2

In [ ]:
# Reading the channel data
df1a = pd.read_csv("Data_channel_matrices/Data_channel_matrices1.csv")
#print(df1)
df1a.shape

In [ ]:
# coverting it into numpy array
X_2_a = df1a.to_numpy()

In [ ]:
X_2_a = X_2_a.reshape((29998,N,N))
#print(X_2)
X_2_a = np.nan_to_num(X_2_a)
#X_2

In [ ]:
# Stacking X_2 above X_2_a
X_2_new = np.concatenate((X_2, X_2_a), axis=0)
X_2_new.shape
if (X_2_new[78094,:,:] == X_2_a[2,:,:]).all():
    print("true")

In [ ]:
X_2 = X_2_new

In [ ]:
# Reading the Source_Destination_info data
df2 = pd.read_csv("Data_Source_Destination_info_matrix(only Src Des).csv")
#print(df2)
df2.shape

In [ ]:
# coverting it into numpy array
X_3 = df2.to_numpy()
#print(X_3)
X_3.shape

In [ ]:
X_3 = X_3.astype(np.float64)
print(X_3.dtype)
#print(X_3)

In [ ]:
# Reading the Spectrum_occupancy data, i.e, NSA
df3 = pd.read_csv("Data_spectrum_matrix.csv")
#print(df3)
df3.shape

In [ ]:
# coverting it into numpy array
X_4 = df3.to_numpy()
#print(X_4)
X_4.shape

# Its already in float64 so no need to convert

In [ ]:
# Reading the output power data
df4 = pd.read_csv("Data_output1.csv")
#print(df4)
df4.shape

In [ ]:
# coverting it into numpy array
X_5 = df4.to_numpy()
#print(X_5)
print(X_5.shape)
X_5 = np.nan_to_num(X_5)

In [ ]:
# This Cell converts each of the (1,30) output of y_data to (1,6)

a=list(range(0, itr))
max1 = 0
count = 0
maxlen = 0
max_path = np.zeros(30)
max_path_ind = 0
X_5_new = np.zeros((itr,6))
for i in a:
    temp = X_5[i]
    maxlen = len(temp[np.nonzero(temp)])
    if maxlen > 6:
        X_5_new[i,:] = temp[0:6]
        temp_non_zero = temp[np.nonzero(temp)]
        X_5_new[i,5] = temp_non_zero[-1]
        continue
    X_5_new[i,:] = temp[0:6]
    #print("original")
    #print(X_5[i])
X_5_new.shape

In [ ]:
# Note: for 33569 itr data  the 150th and 7128 paths have length > 6 , ie, [ 6. 23. 27.  3. 15. 17. 18.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
#  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] and [30.  7.  8. 16.  3. 25. 12. 21.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
#  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

In [ ]:
y_data = X_5_new.astype(np.float64)
print(y_data.dtype)
#print(y_data_temp)
y_data.shape

# No need to unravel it as its already a vector

In [ ]:
X_1.dtype, X_2.dtype, X_3.dtype, X_4.dtype, y_data.dtype

In [ ]:
X_1.shape, X_2.shape, X_3.shape, X_4.shape, y_data.shape

In [ ]:
# scaling parameter for y_data so model gets greater penalty for error and also since output is sparse

y_data = y_data*scaling_parameter

In [ ]:
# converting y_data to tensor

y_data = tf.constant(y_data)
print(y_data.shape)
#y_data = tf.reshape(y_data, [itr,1,1,N*N])
#print(y_data.shape)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.core import Activation
from tensorflow.python.keras.layers.core import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.layers import Flatten
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import concatenate
#from tensorflow.python.keras.optimizers import adam

In [ ]:
# Creating branches of the model

# Activations
a1 = 'relu'
a2 = None

# Branch 1, takes RBI vectors as input and processws it

rbi_input = Input(shape=(N,))
x1 = tf.keras.layers.BatchNormalization()(rbi_input)
#x1 = Dropout(0.3)(x1)
x1 = Dense(50, activation=a1)(x1)
#x1 = Dropout(0.3)(x1)
x1 = Dense(50, activation=a1)(x1)
#x1 = Dense(1000, activation=a2)(x1)
#x1 = Dense(1000, activation=a2)(x1)
out_1 = Dense(N, activation=a1)(x1)

# Branch 2, takes channel matrix as input and processes it

chan_input = Input(shape=(N, N))
x2 = tf.keras.layers.BatchNormalization()(chan_input)
#x2 = Dropout(0.3)(x2)
x2 = Dense(80, activation=a1)(x2)
#x2 = Dropout(0.3)(x2)
#x2 = Flatten()(chan_input)
x2 = Dense(80, activation=a1)(x2)
#x2 = Dense(1000, activation =a2)(x2)
#x2 = Dense(1000, activation =a2)(x2)
x2 = Flatten()(x2)
out_2 = Dense(N*N, activation=a1)(x2)

# Branch 3, takes Source Destination info and processes it

S_D_input = Input(shape=(2, ))
#x3 = Flatten()(S_D_input)
x3 = tf.keras.layers.BatchNormalization()(S_D_input)
#x3 = Dropout(0.3)(x3)
x3 = Dense(10, activation=a1)(x3)
#x3 = Dropout(0.2)(x3)
#x3 = Dropout(0.3)(x3)
x3 = Dense(10, activation=a1)(x3)
#x3 = Dropout(0.2)(x3)
#x3 = Dense(1000, activation =a2)(x3)
#x3 = Dense(1000, activation =a2)(x3)
#x3 = Flatten()(x3)
out_3 = Dense(10, activation=a1)(x3)

# Branch 4, takes Spectrum occupancy info and processes it

Spec_input = Input(shape=(1, ))
#x4 = Flatten()(Spec_input)
x4 = tf.keras.layers.BatchNormalization()(Spec_input)
#x4 = Dropout(0.3)(x4)
x4 = Dense(20, activation=a1)(x4)
#x4 = Dropout(0.3)(x4)
x4 = Dense(20, activation=a1)(x4)
#x4 = Dense(1000, activation =a2)(x4)
#x4 = Dense(1000, activation =a2)(x4)
#x4 = Flatten()(x4)
out_4 = Dense(10, activation=a1)(x4)

# Now we concatenate all the branches 

concatenated = concatenate([out_1, out_2, out_3, out_4]) ## concatenate the four branches
#x5 = Dropout(0.1)(cocatenated)
x5 = Dense(100, activation=a1)(concatenated)
x5 = Dropout(0.3)(x5)
x5 = Dense(100, activation=a1)(x5)
x5 = Dropout(0.3)(x5)
x5 = Dense(100, activation=a1)(x5)
#x5 = Dense(2000, activation=a2)(x5)
#x5 = Dense(2000, activation=a2)(x5)
#x5 = Dense(2000, activation=a2)(x5)
#x5 = Dropout(0.2)(x5)
out = Dense(6, activation=a1)(x5)
model = Model([rbi_input, chan_input, S_D_input, Spec_input], out, name='Model_3')

# Set optimizer

#opt = Adam(lr=0.01, decay=1e-6)
#sgd = SGD(lr=0.0, momentum=0.9)

# Compile the model

#model.compile(loss='mse',
#              optimizer = 'adam',
#              metrics=['accuracy'])

model.compile(loss=tf.keras.losses.mse, # mae is short for mean absolute error
              optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001), # SGD is short for stocastic gradient descent
              metrics=["accuracy"])

#model.compile(loss=tf.keras.losses.mse,
#              optimizer=tf.keras.optimizers.Adam(),
#              metrics=["accuracy"])

# Fit the model

history = model.fit([X_1, X_2, X_3, X_4], y_data, batch_size=64, epochs=2650, validation_split = 0.03)
#next try batch size of 32
#The s2nd dropout gave less accuracy than just the one dropout

Epoch 436/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.4958 - accuracy: 0.9234 - val_loss: 34.7722 - val_accuracy: 0.7000
Epoch 437/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.4263 - accuracy: 0.9237 - val_loss: 34.1300 - val_accuracy: 0.6873
Epoch 438/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.4158 - accuracy: 0.9217 - val_loss: 35.2965 - val_accuracy: 0.6793
Epoch 439/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.5563 - accuracy: 0.9213 - val_loss: 34.9513 - val_accuracy: 0.6796
Epoch 440/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.5129 - accuracy: 0.9213 - val_loss: 33.9729 - val_accuracy: 0.7095
Epoch 441/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.4222 - accuracy: 0.9222 - val_loss: 33.6714 - val_accuracy: 0.7101
Epoch 442/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.5357 - accuracy: 

Epoch 545/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.3956 - accuracy: 0.9231 - val_loss: 33.7407 - val_accuracy: 0.6997
Epoch 546/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.4365 - accuracy: 0.9220 - val_loss: 35.1002 - val_accuracy: 0.6963
Epoch 547/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.3219 - accuracy: 0.9246 - val_loss: 34.1896 - val_accuracy: 0.7012
Epoch 548/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.5069 - accuracy: 0.9213 - val_loss: 34.6318 - val_accuracy: 0.7015
Epoch 549/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.3421 - accuracy: 0.9240 - val_loss: 34.2419 - val_accuracy: 0.7166
Epoch 550/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.4109 - accuracy: 0.9232 - val_loss: 33.6190 - val_accuracy: 0.6984
Epoch 551/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.3986 - accuracy: 

Epoch 654/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.3769 - accuracy: 0.9225 - val_loss: 35.0328 - val_accuracy: 0.6957
Epoch 655/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.3939 - accuracy: 0.9217 - val_loss: 34.1157 - val_accuracy: 0.6883
Epoch 656/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.2935 - accuracy: 0.9234 - val_loss: 34.3257 - val_accuracy: 0.6984
Epoch 657/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.3005 - accuracy: 0.9238 - val_loss: 34.0586 - val_accuracy: 0.7058
Epoch 658/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.3780 - accuracy: 0.9231 - val_loss: 33.7611 - val_accuracy: 0.7080
Epoch 659/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.3621 - accuracy: 0.9235 - val_loss: 34.1833 - val_accuracy: 0.7061
Epoch 660/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.3396 - accuracy: 

Epoch 763/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.4011 - accuracy: 0.9223 - val_loss: 34.7022 - val_accuracy: 0.6867
Epoch 764/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.2324 - accuracy: 0.9249 - val_loss: 35.6907 - val_accuracy: 0.6842
Epoch 765/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.2711 - accuracy: 0.9244 - val_loss: 34.6751 - val_accuracy: 0.6990
Epoch 766/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.3646 - accuracy: 0.9228 - val_loss: 35.0836 - val_accuracy: 0.6879
Epoch 767/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.2746 - accuracy: 0.9259 - val_loss: 34.4277 - val_accuracy: 0.7142
Epoch 768/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.2808 - accuracy: 0.9247 - val_loss: 34.3109 - val_accuracy: 0.7120
Epoch 769/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.3373 - accuracy: 

Epoch 872/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.4973 - accuracy: 0.9232 - val_loss: 35.2538 - val_accuracy: 0.6867
Epoch 873/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.5921 - accuracy: 0.9214 - val_loss: 35.6451 - val_accuracy: 0.6950
Epoch 874/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.5390 - accuracy: 0.9230 - val_loss: 35.7388 - val_accuracy: 0.6799
Epoch 875/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.5419 - accuracy: 0.9212 - val_loss: 35.4780 - val_accuracy: 0.6923
Epoch 876/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.6285 - accuracy: 0.9203 - val_loss: 34.4811 - val_accuracy: 0.6987
Epoch 877/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.5833 - accuracy: 0.9203 - val_loss: 35.1043 - val_accuracy: 0.6941
Epoch 878/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.4998 - accuracy: 

...

1639/1639 [==============================] - 15s 9ms/step - loss: 3.5812 - accuracy: 0.9165 - val_loss: 36.5945 - val_accuracy: 0.6565
Epoch 2170/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.9309 - accuracy: 0.9143 - val_loss: 34.9958 - val_accuracy: 0.6904
Epoch 2171/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.5728 - accuracy: 0.9168 - val_loss: 35.2246 - val_accuracy: 0.6716
Epoch 2172/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.6266 - accuracy: 0.9179 - val_loss: 34.6469 - val_accuracy: 0.6750
Epoch 2173/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.4852 - accuracy: 0.9179 - val_loss: 34.6941 - val_accuracy: 0.6701
Epoch 2174/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.6170 - accuracy: 0.9162 - val_loss: 36.8263 - val_accuracy: 0.6648
Epoch 2175/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.5053 - accuracy: 0.9174 - 

1639/1639 [==============================] - 15s 9ms/step - loss: 3.5235 - accuracy: 0.9180 - val_loss: 36.5041 - val_accuracy: 0.6722
Epoch 2224/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.6771 - accuracy: 0.9179 - val_loss: 38.4723 - val_accuracy: 0.6408
Epoch 2225/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.4793 - accuracy: 0.9183 - val_loss: 36.6981 - val_accuracy: 0.6694
Epoch 2226/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.4390 - accuracy: 0.9188 - val_loss: 35.7751 - val_accuracy: 0.6664
Epoch 2227/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.6517 - accuracy: 0.9160 - val_loss: 35.4166 - val_accuracy: 0.6738
Epoch 2228/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.5347 - accuracy: 0.9179 - val_loss: 38.6277 - val_accuracy: 0.6247
Epoch 2229/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.6401 - accuracy: 0.9177 - 

1639/1639 [==============================] - 15s 9ms/step - loss: 3.7031 - accuracy: 0.9147 - val_loss: 37.2996 - val_accuracy: 0.6503
Epoch 2332/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.8109 - accuracy: 0.9146 - val_loss: 36.8925 - val_accuracy: 0.6580
Epoch 2333/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.6397 - accuracy: 0.9162 - val_loss: 35.5317 - val_accuracy: 0.6802
Epoch 2334/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.6836 - accuracy: 0.9161 - val_loss: 35.7388 - val_accuracy: 0.6870
Epoch 2335/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.5605 - accuracy: 0.9179 - val_loss: 36.4676 - val_accuracy: 0.6512
Epoch 2336/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.5533 - accuracy: 0.9168 - val_loss: 38.2649 - val_accuracy: 0.6405
Epoch 2337/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.7409 - accuracy: 0.9153 - 

1639/1639 [==============================] - 15s 9ms/step - loss: 3.5869 - accuracy: 0.9157 - val_loss: 36.1456 - val_accuracy: 0.6750
Epoch 2440/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.8169 - accuracy: 0.9138 - val_loss: 37.3127 - val_accuracy: 0.6466
Epoch 2441/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.7989 - accuracy: 0.9139 - val_loss: 36.1040 - val_accuracy: 0.6682
Epoch 2442/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.9936 - accuracy: 0.9112 - val_loss: 36.5913 - val_accuracy: 0.6593
Epoch 2443/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.7687 - accuracy: 0.9130 - val_loss: 35.5449 - val_accuracy: 0.7000
Epoch 2444/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.5523 - accuracy: 0.9165 - val_loss: 35.2505 - val_accuracy: 0.6873
Epoch 2445/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.7432 - accuracy: 0.9149 - 

1639/1639 [==============================] - 15s 9ms/step - loss: 3.9101 - accuracy: 0.9122 - val_loss: 36.9845 - val_accuracy: 0.6648
Epoch 2548/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 4.0213 - accuracy: 0.9118 - val_loss: 37.5308 - val_accuracy: 0.6602
Epoch 2549/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.7169 - accuracy: 0.9154 - val_loss: 37.2714 - val_accuracy: 0.6642
Epoch 2550/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.6318 - accuracy: 0.9141 - val_loss: 36.5957 - val_accuracy: 0.6673
Epoch 2551/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.7939 - accuracy: 0.9146 - val_loss: 34.3978 - val_accuracy: 0.6963
Epoch 2552/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.7616 - accuracy: 0.9163 - val_loss: 35.2653 - val_accuracy: 0.6781
Epoch 2553/2650
1639/1639 [==============================] - 15s 9ms/step - loss: 3.7236 - accuracy: 0.9168 - 

In [ ]:
model.summary()

In [ ]:
# Now we read the test data to evaluate our model


# Reading the Test RBI data
df_test = pd.read_csv("Test_Data_rbi_vectors.csv")
#print(df_test)
df_test.shape

In [ ]:
# coverting it into numpy array
tem_test = df_test.to_numpy()
#print(tem_test)
tem_test.shape
X_1_test = tem_test

In [ ]:
# Reading the Test channel data
df1_test = pd.read_csv("Test_Data_channel_matrices.csv")
#print(df1_test)
df1_test.shape

In [ ]:
# coverting it into numpy array
X_2_test = df1_test.to_numpy()
X_2_test.shape

In [ ]:
X_2_test = X_2_test.reshape((itr_test,N,N))
#print(X_2_test)
X_2_test = np.nan_to_num(X_2_test)
#X_2_test

In [ ]:
# Reading the Test Source_Destination_info data

df2_test = pd.read_csv("Test_Data_Source_Destination_info_matrix(only Src Des).csv")
#print(df2_test)
df2_test.shape

In [ ]:
# coverting it into numpy array
X_3_test = df2_test.to_numpy()
#print(X_3_test)
X_3_test.shape

In [ ]:
X_3_test = X_3_test.astype(np.float64)
print(X_3_test.dtype)
#print(X_3_test)

In [ ]:
# Reading the Test Spectrum_occupancy data, i.e, NSA

df3_test = pd.read_csv("Test_Data_spectrum_matrix.csv")
#print(df3_test)
df3_test.shape

In [ ]:
# coverting it into numpy array
X_4_test = df3_test.to_numpy()
#print(X_4_test)
X_4_test.shape

In [ ]:
X_4_test = X_4_test.astype(np.float64)
print(X_4_test.dtype)
#print(X_4_test)

In [ ]:
# Reading the Test output power data

df4_test = pd.read_csv("Test_Data_output1.csv")
#print(df4_test)
df4_test.shape

In [ ]:
# coverting it into numpy array
X_5_test = df4_test.to_numpy()
#print(X_5_test)
print(X_5_test.shape)
X_5_test = np.nan_to_num(X_5_test)

In [ ]:
# This Cell converts each of the (1,30) y_data_test output to (1,6)

a=list(range(0, itr_test))
max1 = 0
count = 0
maxlen = 0
max_path = np.zeros(30)
max_path_ind = 0
X_5_test_new = np.zeros((itr_test,6))
for i in a:
    temp = X_5_test[i]
    maxlen = len(temp[np.nonzero(temp)])
    if maxlen > 6:
        X_5_test_new[i,:] = temp[0:6]
        temp_non_zero = temp[np.nonzero(temp)]
        X_5_test_new[i,5] = temp_non_zero[-1]
        continue
    X_5_test_new[i,:] = temp[0:6]
    #print("original")
    #print(X_5[i])
X_5_test_new.shape

In [ ]:
y_data_test = X_5_test_new.astype(np.float64)
print(y_data_test.dtype)
#print(y_data_test)
y_data_test.shape

In [ ]:
X_1_test.dtype, X_2_test.dtype, X_3_test.dtype, X_4_test.dtype, y_data_test.dtype

In [ ]:
X_1_test.shape, X_2_test.shape, X_3_test.shape, X_4_test.shape, y_data_test.shape

In [ ]:
X_data_test = [X_1_test, X_2_test, X_3_test, X_4_test]

In [ ]:
# converting y_data_test to tensor

y_data_test = tf.constant(y_data_test)
print(y_data_test.shape)
#y_data_test = tf.reshape(y_data_test, [itr_test,1,1,N*N])
#print(y_data_test.shape)
y_data_test

In [ ]:
# We scale the non zero values of Test output power so that there is some sizable error if the model outputs a zero matrix
#scaling_parameter = 100

y_data_test = y_data_test*scaling_parameter
y_data_test

In [ ]:
# Now we make predictions using our model
import time

t = time.perf_counter()
y_pred = model.predict(X_data_test)
elapsed = time.perf_counter() - t
print(elapsed)
print(y_pred.shape)
print("For 1 prediction")
print(elapsed/itr_test)

In [ ]:
# Evaluate the model on the test

t1 = time.perf_counter()
model.evaluate(X_data_test, y_data_test)
elapsed1 = time.perf_counter() - t1
print(elapsed1)

In [ ]:
x_axis = np.arange(1,(6)+1)
x_axis.shape
ind = 3
print(y_pred[ind]/scaling_parameter)
y_data_test[ind]/scaling_parameter

In [ ]:
a = list(range(0, itr_test))
#for i in a:
#    plt.figure()
#    plt.plot(x_axis,y_data_test[i]/scaling_parameter,'b',x_axis,y_pred[i]/scaling_parameter,'r')
#plt.show()

In [ ]:
# summarize history for accuracy
plt.ylim((0.2, 1))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
#plt.ylim((20, 40))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save('saved_model/model_(17-09-2023)')


In [ ]:
#a = list(range(0, 100))
#x_axis = np.array(x_axis)
#for i in a:
#    plt.figure()
#    plt.stem(x_axis, np.array(y_data_test[i]/scaling_parameter),'b')
#    plt.stem( x_axis, np.array(y_pred[i]/scaling_parameter), 'r', markerfmt ='D')
#    plt.legend(['Actual Value', 'Predicted Value'], loc='upper right')

#plt.show()
